In [1]:
from __future__ import print_function

import numpy as np

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from keras.applications import vgg19
from keras.layers import Dense, Input
from keras.models import Sequential, Model
import tensorflow as tf

# Set backend as tensorflow
from keras import backend as K
K.set_image_dim_ordering("tf")

from PIL import Image

%matplotlib inline

Using TensorFlow backend.


In [2]:
# Reset the graph
tf.reset_default_graph()

# Start interactive session
sess = tf.InteractiveSession()
K.set_session(sess)

In [3]:
import h5py

f = h5py.File("vgg19_weights_tf_dim_ordering_tf_kernels.h5", "r")
print(f.keys())

[u'block1_conv1', u'block1_conv2', u'block1_pool', u'block2_conv1', u'block2_conv2', u'block2_pool', u'block3_conv1', u'block3_conv2', u'block3_conv3', u'block3_conv4', u'block3_pool', u'block4_conv1', u'block4_conv2', u'block4_conv3', u'block4_conv4', u'block4_pool', u'block5_conv1', u'block5_conv2', u'block5_conv3', u'block5_conv4', u'block5_pool', u'fc1', u'fc2', u'flatten', u'predictions']


In [4]:
class CONFIG:
    IMAGE_WIDTH = 224
    IMAGE_HEIGHT = 224
    COLOR_CHANNELS = 3
    CONTENT_WEIGHT = 0.025
    STYLE_WEIGHT = 5.0
    TOTAL_VARIATION_WEIGHT = 1.0
    MEANS = np.array([123.68, 116.779, 103.939]).reshape((1,1,1,3))

In [5]:
def reshape_and_normalize_image(image_path):
    """
    Reshape and normalize the input image (content or style)
    """
    
    image = Image.open(image_path)
    image = image.resize((CONFIG.IMAGE_WIDTH, CONFIG.IMAGE_HEIGHT))
    
    image_array = np.asarray(image, dtype='float32')
    image = np.expand_dims(image_array, axis=0)
    
    # Substract the mean to match the expected input of VGG16
    image = image - CONFIG.MEANS
    
    return image

In [6]:
# Get content_image from file
content_image_path = "./images/monet.jpg"
content_image = reshape_and_normalize_image(content_image_path)

In [25]:
style_image_path = "./images/louvre_small.jpg"
style_image = reshape_and_normalize_image(style_image_path)

In [27]:
# Generate a random noise_image
generated_image = np.random.uniform(-20, 20, (1, CONFIG.IMAGE_HEIGHT, CONFIG.IMAGE_WIDTH, CONFIG.COLOR_CHANNELS)) \
                    .astype('float32')

# Set the input_image to be a weighted average of the content_image and a noise_image
generated_image = generated_image * 0.6 + content_image * (1 - 0.6)

In [17]:
input_tensor = Input((224, 224, 3))

In [18]:
model = vgg19.VGG19(weights=None, input_tensor=input_tensor, include_top=True)
model.load_weights("vgg19_weights_tf_dim_ordering_tf_kernels.h5")

In [19]:
dense_outputs = Dense(2, activation='softmax')(model.layers[-2].output)

In [29]:
#Create your own model 
prediction_model = Model(inputs=model.input, outputs=dense_outputs)
prediction_model.predict(generated_image)

#In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training
# prediction_model.summary()

array([[ 0.17668602,  0.82331401]], dtype=float32)

In [31]:
prediction_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
data = np.random.random((1000, 224, 224, 3))
labels = np.random.randint(2, size=(1000, 2))
prediction_model.fit(data, labels, epochs=10, batch_size=32)

Epoch 1/10
